In [1]:
import os
os.chdir("../")
os.getcwd()

'/home/izam/coding/hackathon'

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PromptGenerationConfig:
    root_dir: Path
    data_path: Path
    visualization_path: Path
    data_information_file: Path
    prompt_file_name: str

In [3]:
from gemInsights.constants import *
from gemInsights.utils.common import read_yaml, create_directories

In [4]:
#udpate configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])


    def get_prompt_generation_config(self) -> PromptGenerationConfig:
        config = self.config.prompt_generation
        
        create_directories([config.root_dir])

        prompt_generation_config = PromptGenerationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            visualization_path=config.visualization_path,
            data_information_file=config.data_information_file,
            prompt_file_name=config.prompt_file_name,
        )

        return prompt_generation_config

In [5]:
import pandas as pd
from gemInsights.utils.common import load_json, save_bin
import os
import base64
from gemInsights import logger
from vertexai.preview.generative_models import Part

In [6]:
class PromptGeneration:
    def __init__(self, config: PromptGenerationConfig):
        self.config = config

    def generate(self):
        df = pd.read_csv(self.config.data_path)
        info_json = load_json(Path(self.config.data_information_file))
        target_col = info_json.target_col
        # additional_info = info_json.additional_info
        # df_shape = df.shape
        # df_columns = list(df.columns)
        # df_describe = str(df.describe())+"\n"+ str(df.describe(include=["O"]))
        main_prompt = "Act as a data analyst. Here is the complete information and visualization images of a dataset. give valuable insights from the data point wise"
        images = []
        image_dir = os.path.join(self.config.visualization_path, target_col)
        image_files = os.listdir(image_dir)
        for image_file in image_files:
            image_path = os.path.join(image_dir, image_file)
            img = open(image_path, "rb").read()
            img_bytes = Part.from_data(base64.b64decode(base64.encodebytes(img)), mime_type="image/jpeg")
            logger.info(f"added the image - {image_file}")
            images.append(img_bytes)

        prompt = [
            main_prompt,
            # f"This is the target column of the dataset - '{target_col}'",
            # f"Here are some of the informations related to the dataset - '{additional_info}'",
            # f"The shape of the dataset is {df_shape}",
            # f"The columns in the dataset are {df_columns}",
            # f"Here are some of the general statistics related the dataset - {df_describe}",
        ]

        result = prompt +images

        save_bin(data=result, path=Path(os.path.join(self.config.root_dir, self.config.prompt_file_name)))
        
        

In [7]:
try:
    config = ConfigurationManager()
    prompt_generation_config = config.get_prompt_generation_config()
    prompt_generation = PromptGeneration(config=prompt_generation_config)
    prompt_generation.generate()
except Exception as e:
    raise e

[2023-12-21 13:42:59,579: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-21 13:42:59,580: INFO: common: created directory at: artifacts]
[2023-12-21 13:42:59,581: INFO: common: created directory at: artifacts/prompt_generation]
[2023-12-21 13:42:59,585: INFO: common: json file loaded succesfully from: artifacts/data_ingestion/data_info.json]
[2023-12-21 13:42:59,588: INFO: 2811190944: added the image - Dist_Plots_Numerics.jpg]
[2023-12-21 13:42:59,589: INFO: 2811190944: added the image - Scatter_Plots.jpg]
[2023-12-21 13:42:59,590: INFO: 2811190944: added the image - Box_Plots.jpg]
[2023-12-21 13:42:59,593: INFO: 2811190944: added the image - Bar_Plots.jpg]
[2023-12-21 13:42:59,595: INFO: 2811190944: added the image - Pair_Scatter_Plots.jpg]
[2023-12-21 13:42:59,596: INFO: 2811190944: added the image - Dist_Plots_target.jpg]
[2023-12-21 13:42:59,599: INFO: common: binary file saved at: artifacts/prompt_generation/prompt.joblib]
